In [ ]:
import os, sys, import_ipynb, ipynb
if os.getcwd() == 'C:\\Users\\admin\\Desktop\\retail_data_analysis\\code':
    print ("Already in code directory")
else:
    os.chdir('..//code')
        
# import python libraries
from importLibraries import *


def fun_h2oRegression():
    
    h2o.init()
    os.chdir(os.path.expanduser('../input'))
    # Import training set 
    rawdata_marketplace = pd.read_csv('input_data_preProcessed.csv')
    #rawdata_marketplace = h2o.import_file("Online_Retail.xlsx")
    
    rawdata_marketplace = rawdata_marketplace[['UnitPrice', 'CustomerID', 'Country', 'new_Only_InvoiceDate', 'new_Only_InvoiceHour', 'revenue', 'noun_1', 'nearestConcept']]
        
    
    
    na_list = ['NA', 'none', 'nan', 'NaT']
    col_dtypes = {'UnitPrice':'numeric', 'CustomerID':'string', 'Country': 'string', 'new_Only_InvoiceDate': 'string', 'new_Only_InvoiceHour': 'string', 'revenue': 'numeric', 'noun_1': 'string', 'nearestConcept': 'string'}
    rawdata_marketplace = h2o.H2OFrame(rawdata_marketplace, column_types= col_dtypes, na_strings=na_list)
    
    #display(rawdata_marketplace.describe())
    
    # Split the data into Train/Test/Validation with Train having 70% and test and validation 15% each    
    rawdata_marketplace_train,rawdata_marketplace_test,rawdata_marketplace_valid = rawdata_marketplace.split_frame(ratios=[.7, .15])
    display(rawdata_marketplace_train.shape, rawdata_marketplace_test.shape, rawdata_marketplace_valid.shape)
    
    display(rawdata_marketplace_train.head())
    
    
    predictors = list(rawdata_marketplace.columns) 
    predictors.remove('revenue')  
    display(predictors)
    
    aml = H2OAutoML(max_models = 5, max_runtime_secs=500)
    
    aml.train(x=predictors, y='revenue', training_frame=rawdata_marketplace_train, validation_frame=rawdata_marketplace_test)

    lb = aml.leaderboard
    display(lb)
    
    os.chdir('..//output')
    # Get a model
    m = h2o.get_model(lb[4,"model_id"])
    m.varimp(True).to_csv('H2O_featureimportances.csv')
    #m.varimp_plot()
    m.std_coef_plot()
    
    preds = aml.leader.predict(rawdata_marketplace_valid)
    display(preds)
    #display(rawdata_marketplace_valid.head(10))
    
    
    